In [1]:
# model building: Naive Bayes, KNN model

In [2]:
# Import our libraries 

# Pandas and numpy for data wrangling
import pandas as pd
import numpy as np

# Seaborn / matplotlib for visualization 
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# Import the trees from sklearn
from sklearn import tree

# Helper function to split our data
from sklearn.model_selection import train_test_split

# Importing the Porter Stemmer for text stemming
from nltk.stem.porter import PorterStemmer

# Importing the string module for handling special characters
import string

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score

!pip install wordcloud
# Importing WordCloud for text visualization
from wordcloud import WordCloud

# Importing NLTK for natural language processing
import nltk
from nltk.corpus import stopwords    # For stopwords


# Downloading NLTK data
nltk.download('stopwords')   # Downloading stopwords data
nltk.download('punkt')       # Downloading tokenizer data

! pip install langdetect
from langdetect import detect

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilyliang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lilyliang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
spam_or_not_df = "https://raw.githubusercontent.com/AliceLiu17/csc448_final/lily_branch/data/combined_data_new.csv"

df = pd.read_csv(spam_or_not_df)

In [4]:
df.head()

,Unnamed: 0,label,email
0,0,0,"Go until jurong point, crazy.. Available only ..."
1,1,0,Ok lar... Joking wif u oni...
2,2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,3,0,U dun say so early hor... U c already then say...
4,4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Drop the 'Unnamed: 0' column
df = df.drop('Unnamed: 0', axis=1)

In [6]:
print(df.shape)

# df.dropna(inplace=True)
df = df.dropna()

# Sanity Check
print(df.shape)

# pring how many null values there are
df.isnull().sum()

(8030, 2)
(8029, 2)


label    0
email    0
dtype: int64

In [7]:
df.head()

,label,email
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Apply language detection to each email
df['detected_language'] = df['email'].apply(detect_language)

# Count the number of non-English emails and English emails
non_english_count = len(df[df['detected_language'] != 'en'])
english_count = len(df[df['detected_language'] == 'en'])

# Calculate the ratio
if english_count > 0:
    ratio_non_english_to_english = non_english_count / english_count
    print(f"Ratio of non-English emails to English emails: {ratio_non_english_to_english:.2f}")
else:
    print("No English emails found in the dataset.")

Ratio of non-English emails to English emails: 0.07


In [9]:
# Apply language detection to each email
df['detected_language'] = df['email'].apply(detect_language)

# Filter out non-English emails
df = df[df['detected_language'] == 'en']

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove punctuation and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return " ".join(tokens)


[nltk_data] Downloading package punkt to /Users/lilyliang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilyliang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lilyliang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
df["email"]=df['email'].apply(lambda x : preprocess(x))

In [12]:
df.head()

,label,email,detected_language
0,0,go jurong point crazy available bugis n great ...,en
2,1,free entry wkly comp win fa cup final tkts may...,en
3,0,u dun say early hor u c already say,en
4,0,nah think go usf life around though,en
5,1,freemsg hey darling week word back like fun st...,en


In [13]:
x_train, x_test, y_train, y_test = train_test_split(df.email, df.label, test_size=0.2, random_state=45)

In [14]:
# Naive_Bayes
from sklearn.pipeline import Pipeline
model1 = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Multi NB', MultinomialNB())         
])
model1

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

In [15]:
from sklearn.metrics import classification_report
model1.fit(x_train, y_train)
y_pred1 = model1.predict(x_test)
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1283
           1       0.99      0.36      0.53       222

    accuracy                           0.90      1505
   macro avg       0.94      0.68      0.74      1505
weighted avg       0.91      0.90      0.89      1505



In [16]:
v = model1.named_steps['vectorizer_tfidf']
vocabulary = v.get_feature_names_out()
for word in vocabulary:
    index=v.vocabulary_.get(word)
    idf_score=v.idf_[index]
    print(f"{word} : {idf_score}")

aa : 7.622902533540516
aac : 9.009196894660407
aachen : 9.009196894660407
aah : 8.316049714100462
aalib : 7.910584605992296
aaniye : 9.009196894660407
aanumber : 8.603731786552242
aaron : 7.756433926165038
aathi : 8.603731786552242
aavid : 9.009196894660407
aavishkar : 9.009196894660407
aaxine : 8.603731786552242
ab : 8.092906162786251
abacha : 8.092906162786251
abandon : 7.910584605992296
abandoned : 7.505119497884133
abandoning : 9.009196894660407
abatement : 9.009196894660407
abbey : 8.603731786552242
abbreviation : 9.009196894660407
abc : 8.603731786552242
abdomen : 9.009196894660407
abdominal : 9.009196894660407
abeg : 9.009196894660407
abel : 9.009196894660407
aberdeen : 9.009196894660407
aberrant : 8.603731786552242
aberration : 9.009196894660407
abetted : 9.009196894660407
abetting : 9.009196894660407
abi : 8.603731786552242
abide : 8.603731786552242
abidjan : 8.092906162786251
ability : 5.482836370044245
abiola : 7.304448802421981
abit : 8.603731786552242
abiword : 8.092906162

analysing : 9.009196894660407
analysis : 6.566849859291202
analyst : 6.7066118016663605
analytic : 9.009196894660407
analyze : 8.603731786552242
analyzer : 9.009196894660407
analyzes : 9.009196894660407
analyzing : 7.910584605992296
anand : 9.009196894660407
ananova : 8.092906162786251
anantapur : 8.603731786552242
anarchist : 8.316049714100462
anarchocapitalist : 9.009196894660407
anathema : 8.316049714100462
ancestor : 8.316049714100462
ancestral : 8.316049714100462
ancestry : 9.009196894660407
anchor : 8.092906162786251
ancient : 7.756433926165038
ancillary : 9.009196894660407
andean : 8.316049714100462
ander : 9.009196894660407
anders : 6.994293874118141
anderson : 7.217437425432352
andersontop : 9.009196894660407
andes : 8.603731786552242
andhra : 8.603731786552242
andordered : 9.009196894660407
andorra : 9.009196894660407
andpercentages : 9.009196894660407
andre : 9.009196894660407
andrea : 9.009196894660407
andres : 9.009196894660407
andrew : 7.137394717758815
andrey : 9.0091968

ball : 7.217437425432352
ballantine : 8.603731786552242
baller : 9.009196894660407
ballerina : 9.009196894660407
ballet : 8.603731786552242
ballgame : 9.009196894660407
ballistic : 8.603731786552242
ballmer : 8.603731786552242
balloon : 7.505119497884133
ballot : 8.603731786552242
ballpark : 9.009196894660407
bally : 9.009196894660407
baltimore : 8.092906162786251
bam : 9.009196894660407
bambling : 9.009196894660407
bamboo : 9.009196894660407
bamping : 9.009196894660407
ban : 7.756433926165038
banal : 9.009196894660407
banana : 8.603731786552242
banbury : 9.009196894660407
band : 7.505119497884133
bandage : 9.009196894660407
banded : 8.603731786552242
bandied : 9.009196894660407
bandit : 8.603731786552242
bandwidth : 6.370139565045148
bane : 9.009196894660407
bang : 7.063286745605093
bangalore : 9.009196894660407
bangb : 9.009196894660407
bangbabes : 9.009196894660407
bangkok : 8.316049714100462
bangladesh : 9.009196894660407
banjo : 9.009196894660407
bank : 5.559209348828819
banker : 

bled : 9.009196894660407
bleed : 9.009196894660407
bleeding : 7.910584605992296
bleh : 9.009196894660407
blend : 8.316049714100462
blendage : 9.009196894660407
blending : 8.603731786552242
bless : 7.137394717758815
blessed : 7.756433926165038
blessing : 7.910584605992296
bletchley : 8.316049714100462
blew : 7.756433926165038
blib : 9.009196894660407
blight : 8.603731786552242
blimey : 9.009196894660407
blind : 6.811972317324187
blindingly : 9.009196894660407
blindly : 7.756433926165038
blindness : 9.009196894660407
blinkered : 9.009196894660407
bliss : 7.756433926165038
blissful : 8.603731786552242
blithered : 9.009196894660407
bloat : 8.316049714100462
blob : 8.603731786552242
bloc : 8.603731786552242
block : 6.236608172420625
blockade : 8.603731786552242
blockage : 8.316049714100462
blockbuster : 9.009196894660407
blocked : 7.399758982226306
blockhead : 9.009196894660407
blocking : 7.505119497884133
blocklists : 9.009196894660407
blog : 6.370139565045148
blogchannel : 9.0091968946604

casting : 7.910584605992296
castle : 7.756433926165038
castleford : 9.009196894660407
castletown : 9.009196894660407
castor : 9.009196894660407
casual : 9.009196894660407
casualty : 7.622902533540516
cat : 6.657821637496928
catacomb : 9.009196894660407
catalog : 6.92975535298057
cataloged : 9.009196894660407
catalogue : 8.316049714100462
catalyst : 8.316049714100462
catamite : 9.009196894660407
catastrophe : 8.316049714100462
catastrophic : 8.316049714100462
catch : 5.770518442496026
catching : 7.399758982226306
catchup : 8.092906162786251
catchy : 9.009196894660407
catdoc : 9.009196894660407
catego : 9.009196894660407
categorically : 9.009196894660407
categorized : 8.092906162786251
categorizing : 8.603731786552242
category : 6.5242902448724065
categorynumber : 9.009196894660407
cater : 8.603731786552242
catering : 8.603731786552242
cathartic : 9.009196894660407
cathedral : 8.316049714100462
catholic : 7.756433926165038
catholicism : 8.603731786552242
cato : 8.603731786552242
cattle :

...

squid : 9.009196894660407
squiggle : 8.603731786552242
squirm : 8.316049714100462
sr : 8.316049714100462
src : 6.301146693558197
srclist : 8.316049714100462
srclists : 8.603731786552242
sri : 9.009196894660407
srl : 8.603731786552242
srpm : 8.092906162786251
srpms : 8.603731786552242
srsly : 9.009196894660407
srt : 9.009196894660407
srv : 9.009196894660407
srvnumber : 9.009196894660407
sry : 8.603731786552242
sseldorf : 9.009196894660407
ssh : 6.92975535298057
ssinumberusa : 9.009196894660407
ssl : 7.505119497884133
ssnumber : 7.910584605992296
ssr : 8.316049714100462
st : 6.611301621862036
stab : 8.316049714100462
stabbed : 8.603731786552242
stability : 6.92975535298057
stabilize : 8.316049714100462
stabilized : 9.009196894660407
stable : 6.811972317324187
stacey : 9.009196894660407
stack : 7.756433926165038
stackguard : 9.009196894660407
stadium : 9.009196894660407
staff : 6.335048245233878
staffed : 9.009196894660407
stafford : 9.009196894660407
stage : 6.811972317324187
staggering 

talebones : 9.009196894660407
talent : 7.910584605992296
talented : 7.622902533540516
talentless : 9.009196894660407
talib : 9.009196894660407
taliban : 7.399758982226306
talios : 7.505119497884133
talk : 4.394076377819147
talkbut : 9.009196894660407
talked : 7.622902533540516
talkin : 9.009196894660407
talking : 5.307894920547913
talkington : 9.009196894660407
tall : 6.657821637496928
tallahassee : 9.009196894660407
tallest : 9.009196894660407
tambien : 9.009196894660407
tamilnadu : 9.009196894660407
taming : 9.009196894660407
tampa : 7.505119497884133
tanaka : 9.009196894660407
tandem : 9.009196894660407
tang : 9.009196894660407
tangent : 9.009196894660407
tangled : 8.603731786552242
tank : 7.137394717758815
tanne : 8.603731786552242
tanniel : 9.009196894660407
tanpalaupanamapapua : 9.009196894660407
tantra : 9.009196894660407
tantrum : 9.009196894660407
tanuki : 9.009196894660407
tanzania : 9.009196894660407
tanzanian : 9.009196894660407
tao : 8.316049714100462
taoist : 9.0091968946

transylvania : 9.009196894660407
trap : 7.505119497884133
trapped : 8.092906162786251
trapper : 9.009196894660407
trapping : 9.009196894660407
trash : 7.910584605992296
trauma : 9.009196894660407
traumatic : 8.603731786552242
traumatize : 9.009196894660407
trav : 9.009196894660407
travail : 8.603731786552242
travel : 6.657821637496928
traveled : 8.603731786552242
traveler : 7.910584605992296
traveling : 8.316049714100462
travelled : 8.603731786552242
traveller : 8.316049714100462
travelling : 8.092906162786251
trawl : 9.009196894660407
tray : 9.009196894660407
trck : 9.009196894660407
tre : 9.009196894660407
treacle : 8.316049714100462
tread : 9.009196894660407
treadmill : 8.316049714100462
trearum : 9.009196894660407
treasure : 7.505119497884133
treasury : 7.756433926165038
treat : 6.566849859291202
treated : 7.063286745605093
treatin : 9.009196894660407
treating : 7.910584605992296
treatise : 8.316049714100462
treatment : 6.92975535298057
treaty : 7.756433926165038
treble : 8.6037317

va : 7.304448802421981
vaan : 8.316049714100462
vaanumber : 8.603731786552242
vacant : 8.603731786552242
vacate : 9.009196894660407
vacated : 9.009196894660407
vacation : 7.063286745605093
vaccinating : 9.009196894660407
vaccine : 9.009196894660407
vacuous : 9.009196894660407
vacuum : 8.092906162786251
vader : 9.009196894660407
vagina : 9.009196894660407
vague : 7.756433926165038
vaguely : 8.316049714100462
vain : 9.009196894660407
vainglorious : 9.009196894660407
vajpayee : 9.009196894660407
valdis : 7.622902533540516
vale : 8.603731786552242
valedictorian : 9.009196894660407
valen : 9.009196894660407
valencia : 9.009196894660407
valenti : 8.603731786552242
valentine : 7.217437425432352
valeriana : 9.009196894660407
valhalla : 6.0387824290907055
valid : 5.770518442496026
validate : 7.756433926165038
validated : 8.316049714100462
validation : 8.092906162786251
validator : 8.603731786552242
validity : 8.603731786552242
valign : 8.603731786552242
valium : 9.009196894660407
valley : 7.505

wholesale : 7.756433926165038
wholly : 7.505119497884133
whoo : 9.009196894660407
whoopee : 9.009196894660407
whopping : 9.009196894660407
whore : 8.316049714100462
whorl : 9.009196894660407
whoromatic : 9.009196894660407
whose : 5.895681585450032
whr : 8.603731786552242
whrere : 9.009196894660407
whump : 7.756433926165038
why : 9.009196894660407
wi : 7.756433926165038
wicked : 8.603731786552242
wicket : 9.009196894660407
wicklow : 9.009196894660407
wid : 7.399758982226306
wide : 6.064757915493966
widely : 6.7066118016663605
widening : 7.756433926165038
wider : 8.316049714100462
widespread : 7.063286745605093
widget : 8.092906162786251
widow : 7.399758982226306
width : 7.622902533540516
wielded : 8.603731786552242
wielding : 9.009196894660407
wierd : 8.603731786552242
wif : 6.811972317324187
wife : 5.852196473510293
wifeand : 9.009196894660407
wifi : 7.304448802421981
wiggle : 8.603731786552242
wihtuot : 9.009196894660407
wiki : 9.009196894660407
wil : 6.994293874118141
wild : 7.137394

In [17]:
y_pred1

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
model1.predict(["Congratulations! You've won a free vacation to an exotic island. Claim your prize now by clicking on the link below: [link]"])

array([1])

In [19]:
model1.predict(["Hello Richard, please send in your report EOD."])

array([0])

In [20]:
x_train, x_test, y_train, y_test = train_test_split(df.email, df.label, test_size=0.2, random_state=45)

In [21]:
model2 = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('KNN', KNeighborsClassifier())         
])
model2

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('KNN', KNeighborsClassifier())])

In [22]:
model2.fit(x_train, y_train)
y_pred2 = model2.predict(x_test)
print(classification_report(y_test, y_pred2))

AttributeError: 'NoneType' object has no attribute 'split'